<a href="https://colab.research.google.com/github/MariaTaskin01/Accelally/blob/main/LSTM_SLT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
!pip install tensorflow pandas scikit-learn nltk rouge-score


In [8]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional
from tensorflow.keras.optimizers import Adam
import nltk
from nltk.translate.bleu_score import sentence_bleu
from rouge_score import rouge_scorer

# Load your dataset
dataset = pd.read_csv('/content/preprocessed_data.csv')

# Display the first few rows of the dataset to ensure it's loaded correctly
dataset.head()


FileNotFoundError: [Errno 2] No such file or directory: '/content/Selected Data.csv'

In [9]:
# Define the tokenizer for gloss tokens
gloss_tokenizer = Tokenizer()
gloss_tokenizer.fit_on_texts(dataset['gloss_tokens'].values)

# Convert gloss tokens to sequences
X = gloss_tokenizer.texts_to_sequences(dataset['gloss_tokens'].values)

# Padding the sequences to have the same length
max_sequence_length = max(len(sequence) for sequence in X)
X = pad_sequences(X, maxlen=max_sequence_length, padding='post')

# Define the tokenizer for text
text_tokenizer = Tokenizer()
text_tokenizer.fit_on_texts(dataset['text'].values)

# Convert text to sequences
y = text_tokenizer.texts_to_sequences(dataset['text'].values)

# Padding the sequences to have the same length as the input sequences
y = pad_sequences(y, maxlen=max_sequence_length, padding='post')

# Reshape y to be compatible with sparse_categorical_crossentropy
y = np.expand_dims(y, -1)

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [10]:
VOCAB_SIZE = len(text_tokenizer.word_index) + 1  # Size of the vocabulary for the target sequences
EMBEDDING_DIM = 128  # Dimension of the embedding layer

# Calculate the number of unique tokens in the target sequences
num_output_tokens = VOCAB_SIZE

# Define the model
model = Sequential()
model.add(Embedding(input_dim=len(gloss_tokenizer.word_index) + 1, output_dim=EMBEDDING_DIM, input_length=max_sequence_length))
model.add(Bidirectional(LSTM(128, return_sequences=True)))
model.add(Bidirectional(LSTM(128, return_sequences=True)))
model.add(Dense(num_output_tokens, activation='softmax'))  # Change output dimension

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Display the model summary
model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 44, 128)           1939584   
                                                                 
 bidirectional (Bidirection  (None, 44, 256)           263168    
 al)                                                             
                                                                 
 bidirectional_1 (Bidirecti  (None, 44, 256)           394240    
 onal)                                                           
                                                                 
 dense (Dense)               (None, 44, 20580)         5289060   
                                                                 
Total params: 7886052 (30.08 MB)
Trainable params: 7886052 (30.08 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [1]:
history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)

NameError: name 'model' is not defined